In [1]:
#import used packages
import numpy as np
import pickle # save and load binary files (data, model)

# Set jupyter display in full screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# function to import and export data from cPickle format
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def save_obj(obj, name):
    with open('export/' + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [3]:
#import data
features = unpickle('data/Sigma_features.pkl')
label = unpickle('data/Sigma_labels.pkl')
m = label.shape[0]
print(m)
label = label.reshape(-1,1)

238


In [4]:
# Add bias column (1 vector)
bias = np.ones(features.shape)
features = np.column_stack([features, bias])
features.shape

(238, 64)

In [5]:
# Create parameter vector
n = features.shape[1]
print("Number of features (eg number of parameters): {}".format(n))

parameters = np.random.rand(n,1)
parameters.shape

Number of features (eg number of parameters): 64


(64, 1)

In [6]:
# Compute our hypothesis model (linear regression), use a fonction:

def hypothesis(x, theta):
    return np.dot(x, theta)

predictions = hypothesis(features, parameters)
predictions.shape

(238, 1)

In [7]:
# Fonction de coût
def costFunction(yhat, y):
    return np.square(yhat - y).sum() / (2*y.shape[0])

costFct = costFunction(predictions, label)
costFct

15193727.217210574

In [8]:
# Dérivée de la fonction de coût == gradients
def gradients(yhat, y, x):
    return (((yhat - y) * x).sum(axis=0) / x.shape[0]).reshape(x.shape[1],1)

grads = gradients(predictions, label, features)
grads.shape

(64, 1)

In [9]:
# gradient descent: mise à jour des paramètres
alpha = 0.003
def updateParameters(parameters, grads, alpha):
    return parameters - alpha * grads

parameters = updateParameters(parameters, grads, alpha)
parameters.shape

(64, 1)

In [10]:
# fonction pour tester l'évolution de la fonction de coût: vrai = continuer la descente de gradient
predictions = hypothesis(features, parameters)
def testCostFct(yhat, y, prevCostFct, epsilon):
        return np.abs(costFunction(yhat, y) - prevCostFct) >= epsilon*prevCostFct
    
testCostFct(predictions, label, costFct, 1e-5)

True

# On combine le tout dans une boucle

In [11]:
# Initialisation
def initGradDescent(x):
    n = x.shape[1]
    theta = np.random.rand(n,1)
    yhat = hypothesis(x, theta)
    costFct = 0
    epsilon = 1e-5
    alpha = 0.000000003
    count = 0
    return theta, yhat, costFct, epsilon, alpha, count

In [12]:
# On utilise une boucle while

parameters, predictions, costFct, epsilon, alpha, count = initGradDescent(features)
costFctEvol = []
while testCostFct(predictions, label, costFct, epsilon):
    count += 1
    costFct = costFunction(predictions, label)
    grads = gradients(predictions, label, features)
    parameters = updateParameters(parameters, grads, alpha)
    predictions = hypothesis(features, parameters)
    if count % 1000 == 0:
        print('%3i : cost function = {}'.format(costFct) % count)
    costFctEvol.append(costFct)
print("\nFinish: {} steps, cost function = {}".format(count, costFct))

1000 : cost function = 11388.00307675768
2000 : cost function = 8501.924592767793
3000 : cost function = 6835.096927403849
4000 : cost function = 5683.214398029373
5000 : cost function = 4858.22018252839
6000 : cost function = 4262.482603713224
7000 : cost function = 3830.4517929108965
8000 : cost function = 3515.718148643477
9000 : cost function = 3285.122299601059
10000 : cost function = 3114.9346549251827
11000 : cost function = 2988.167667761783
12000 : cost function = 2892.6579074818587
13000 : cost function = 2819.694231285653
14000 : cost function = 2763.0361812905658
15000 : cost function = 2718.211557989418
16000 : cost function = 2682.013762132457
17000 : cost function = 2652.1420882753796

Finish: 17184 steps, cost function = 2647.1953635892755


# Regularization

In [13]:
# Fonction de coût régularisée
def regCostFunction(yhat, y, lmb, theta):
    return costFunction(yhat, y) + lmb/(2*y.shape[0]) * np.square(theta).sum()


lmb = (m * 0.9)/alpha

costFct = costFunction(predictions, label)
print("fonction de coût       = {}".format(costFct))
regCostFct = regCostFunction(predictions, label, lmb, parameters)
print("fonction de coût (reg) = {}".format(regCostFct))

fonction de coût       = 2647.16889265952
fonction de coût (reg) = 3112937282.5249066


In [14]:
# Dérivée de la fonction de coût regularisée
def regGradients(yhat, y, x, lmb, theta):
    return (((yhat - y) * x).sum(axis=0) / x.shape[0]).reshape(x.shape[1],1) + lmb/x.shape[0]*theta

regGrads = regGradients(predictions, label, features, lmb, parameters)
regGrads.shape

(64, 1)

In [15]:
# gradient descent: mise à jour des paramètres (on utilise la même fonction)
alpha = 0.003

parameters = updateParameters(parameters, regGrads, alpha)
parameters.shape

(64, 1)

In [16]:
# fonction pour tester l'évolution de la fonction de coût: vrai = continuer la descente de gradient
predictions = hypothesis(features, parameters)
def testRegCostFct(yhat, y, lmb, theta, prevCostFct, epsilon):
        return np.abs(regCostFunction(yhat, y, lmb, theta) - prevCostFct) >= epsilon*prevCostFct
    
testRegCostFct(predictions, label, lmb, parameters, regCostFct, 1e-5)

True

In [17]:
# Initialisation
def initRegGradDescent(x):
    m = x.shape[0]
    n = x.shape[1]
    theta = np.random.rand(n,1)
    yhat = hypothesis(x, theta)
    costFct = 0
    epsilon = 1e-10
    alpha = 0.000000003
    lmb = (m * 0.2)/alpha
    count = 0
    return theta, yhat, costFct, epsilon, alpha, lmb, count

In [18]:
# On utilise une boucle while

parameters, predictions, costFct, epsilon, alpha, lmb, count = initRegGradDescent(features)
costFctEvol = []
while testRegCostFct(predictions, label, lmb, parameters, costFct, epsilon):
    count += 1
    costFct = regCostFunction(predictions, label, lmb, parameters)
    grads = regGradients(predictions, label, features, lmb, parameters)
    parameters = updateParameters(parameters, grads, alpha)
    predictions = hypothesis(features, parameters)
    if count % 10 == 0:
        print('%3i : cost function = {}'.format(costFct) % count)
    costFctEvol.append(costFct)
print("\nFinish: {} steps, cost function = {}".format(count, costFct))

 10 : cost function = 14190816.813098796
 20 : cost function = 265697.87570084207
 30 : cost function = 107217.4746736408
 40 : cost function = 105397.00100048564
 50 : cost function = 105376.06300248299
 60 : cost function = 105375.82209721797
 70 : cost function = 105375.81932475726

Finish: 71 steps, cost function = 105375.81931313156
